LOAD DATA 

In [1]:
import numpy as np
import math
import os
import matplotlib.pyplot as plt
data_path ='../data/Task 2/sample_1a2bc3012c9defca.npz' 
print(os.path.exists(data_path))
# data_path ='../data/Task 2/sample_1a2bc3012c9defca.npz' 
print(os.getcwd())
print(os.path.exists(data_path))

False
/home/lando/Homework1_Perception/Task 2
False


In [8]:
info = np.load(data_path,allow_pickle=True)
all_agent_trajs        = info['all_agent']    
all_agent_trajs.shape 
print(all_agent_trajs.shape) #[24 agents, 91 timesteps the interval between each timestep is 0.1 second,\
    #10 denotes [center_x, center_y, center_z, boundingbox_x, boundingbox_y, boundingbox_z, heading, vel_x, vel_y, valid] ]
#the valid flag is either 0 or 1. 1 means the valid, 0 means invalid
obj_types              = info['object_type']
lane_polylines         = info['lane']           #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
road_polylines         = info['road_polylines'] #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
crosswalk_polylines    = info['crosswalk']      #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
speed_bump_polylines   = info['speed_bump']     #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
stop_signs_polylines   = info['stop_sign']      #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
drive_way_polylines    = info['drive_way']      #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]

(24, 91, 10)


We select the 11th timestep as the current timestep 

In [9]:
all_agent_current = all_agent_trajs[:,10]
all_agent_current.shape

(24, 10)

In [10]:
#ground truth for the future 1s,2s,3s can be easily derived by :
predict_horizon = 20 #10,20,30
all_gt_future=all_agent_trajs[:,11:11+predict_horizon]
all_gt_future.shape

(24, 20, 10)

Index of target agnet to preict

In [11]:
tracks        = info['predict_list']  
tracks
agents_to_predict = all_agent_trajs[tracks]
agents_to_predict.shape

#10 denotes [center_x, center_y, center_z, boundingbox_x, boundingbox_y, boundingbox_z, heading, vel_x, vel_y, valid] ]
#the valid flag is either 0 or 1. 1 means the valid, 0 means invalid

(8, 91, 10)

The homework should be done on these 8 agents.

Q1.Using constant velocity model to predict the future (1s,2s,3s) trajectories of the target agent and calculate the ADE and FDE.  

In [12]:
# Define function to calculate ADE and FDE
def calculate_errors(predx,predy,tracks_gt_future,predict_horizon):
    ADEs = []
    FDEs = []
    distance = 0
    for i in range(len(agents_to_predict)):
        # initialize the distance
        total_distance = 0.0

        # Calculate the Euclidean distance between the predicted position and the ground truth position at each time step
        for t in range(len(predx[i])):

            distance = np.linalg.norm([predx[i][t] - tracks_gt_future[i][t][0], predy[i][t] - tracks_gt_future[i][t][1]])
            total_distance += distance

        # Calculate ADE
        ADE = total_distance / predict_horizon
        ADE_str = "{:.8f}".format(ADE)
        ADEs.append(ADE_str)

        # Calculate FDE
        FDE = np.linalg.norm([predx[i][-1] - tracks_gt_future[i][-1][0], predy[i][-1] - tracks_gt_future[i][-1][1]])
        FDE_str = "{:.8f}".format(FDE)
        FDEs.append(FDE_str)
    return ADEs, FDEs


In [13]:
# Predict future trajectories using constant velocity model

tracks_gt_future = all_gt_future[tracks]

agents_to_predict = all_agent_trajs[tracks]

# acquire the initial state
lastest_positionX = agents_to_predict[:,10,0]
lastest_positionY = agents_to_predict[:,10,1]

timestep = 0.1
# invalid flag to check the validation
invalid = 0

# predx,predy are used to save the prediction locations
predx = [[] for _ in range(8)]
predy = [[] for _ in range(8)]
# newX, newY are used to save the present prediction locations in the loop
newX = [0] * 8
newY = [0] * 8
# loop for each agent
for num in range(8):
    for t in range(predict_horizon):
        if agents_to_predict[num, 10 + t, -1] == 1:
            if invalid == 0:
                newX[num] = agents_to_predict[num ,10 + t ,7] * timestep + lastest_positionX[num]
                newY[num] = agents_to_predict[num ,10 + t ,8] * timestep + lastest_positionY[num]
                predx[num].append(newX[num])
                predy[num].append(newY[num])
                # renew the new positions
                lastest_positionX[num], lastest_positionY[num] = newX[num], newY[num]

            else:
                newX[num] = agents_to_predict[num ,10 + t - invalid ,7] * timestep * (invalid + 1) + lastest_positionX[num]
                newY[num] = agents_to_predict[num ,10 + t - invalid ,8] * timestep * (invalid + 1) + lastest_positionY[num]
                predx[num].append(newX[num])
                predy[num].append(newY[num])
                lastest_positionX[num], lastest_positionY[num] = newX[num], newY[num]
                invalid = 0
        else:
            invalid += 1

ADEs, FDEs = calculate_errors(predx, predy, tracks_gt_future, predict_horizon)

lastest_positionX = [0] * 8
lastest_positionY = [0] * 8

# Print results
print(f"ADE for {predict_horizon/10}s prediction horizon:", ADEs)
print(f"FDE for {predict_horizon/10}s prediction horizon:", FDEs)
print('\n')

ADE for 2.0s prediction horizon: ['0.01911305', '0.13527665', '0.19310980', '0.11951432', '0.07339844', '0.05430345', '0.06553870', '0.01828897']
FDE for 2.0s prediction horizon: ['0.01856312', '0.26879040', '0.35412209', '0.20801483', '0.18174004', '0.07580127', '0.08833877', '0.01269311']




#### Constant Velocity Model Result
ADE for 1.0s prediction horizon: ['0.01855008', '0.07625426', '0.11207651', '0.07010401', '0.03945416', '0.04476424', '0.05458820', '0.01933917']

FDE for 1.0s prediction horizon: ['0.00914143', '0.11392895', '0.18303247', '0.10324587', '0.03135093', '0.05822754', '0.06435783', '0.02153425']

ADE for 2.0s prediction horizon: ['0.01911305', '0.13527665', '0.19310980', '0.11951432', '0.07339844', '0.05430345', '0.06553870', '0.01828897']

FDE for 2.0s prediction horizon: ['0.01856312', '0.26879040', '0.35412209', '0.20801483', '0.18174004', '0.07580127', '0.08833877', '0.01269311']

ADE for 3.0s prediction horizon: ['0.01853832', '0.23521721', '0.28616008', '0.16878053', '0.13838122', '0.06317096', '0.07150245', '0.01941382']

FDE for 3.0s prediction horizon: ['0.01158061', '0.57718451', '0.55741967', '0.29914092', '0.34613529', '0.09002940', '0.07918140', '0.02856680']

Q2.Using constant acceleration model to predict the future (1s,2s,3s) trajectories of the target agent and calculate the ADE and FDE. 

In [14]:
# Predict future trajectories using constant velocity model
# for predict_horizon in {10,20,30}:
tracks_gt_future = all_gt_future[tracks]
agents_to_predict = all_agent_trajs[tracks]

lastest_positionX = agents_to_predict[:,10,0]
lastest_positionY = agents_to_predict[:,10,1]

timestep = 0.1
invalid = 0

accelerationX = [[] for _ in range(8)]
accelerationY = [[] for _ in range(8)]
# make up the acceleration list
for num in range(8):
    accelerationX[num] = np.diff(agents_to_predict[num, 10 : 10 + predict_horizon + 1, 7])
    accelerationY[num] = np.diff(agents_to_predict[num, 10 : 10 + predict_horizon + 1, 8])


predx = [[] for _ in range(8)]
predy = [[] for _ in range(8)]
newX = [0] * 8
newY = [0] * 8
# loop for each agent
for num in range(8):
    t = 0
    for t in range(predict_horizon):
        if agents_to_predict[num, 10 + t, -1] == 1:
            if invalid == 0:
                newX[num] = agents_to_predict[num ,10 + t ,7] * timestep + lastest_positionX[num] + 0.5 * accelerationX[num][t] * (timestep ** 2)
                newY[num] = agents_to_predict[num ,10 + t ,8] * timestep + lastest_positionY[num] + 0.5 * accelerationY[num][t] * (timestep ** 2)
                predx[num].append(newX[num])
                predy[num].append(newY[num])
                lastest_positionX[num], lastest_positionY[num] = newX[num], newY[num]
            else:
                newX[num] = agents_to_predict[num ,10 + t - invalid ,7] * timestep * (invalid + 1) + lastest_positionX[num] + 0.5 * accelerationX[num][t] * (timestep * (invalid + 1)) ** 2
                newY[num] = agents_to_predict[num ,10 + t - invalid ,8] * timestep * (invalid + 1) + lastest_positionY[num] + 0.5 * accelerationY[num][t] * (timestep * (invalid + 1)) ** 2
                predx[num].append(newX[num])
                predy[num].append(newY[num])
                lastest_positionX[num], lastest_positionY[num] = newX[num], newY[num]
                invalid = 0
        else:
            invalid += 1

ADEs, FDEs = calculate_errors(predx, predy, tracks_gt_future, predict_horizon)

# Print results
print(f"ADE for {predict_horizon/10}s prediction horizon:", ADEs)
print(f"FDE for {predict_horizon/10}s prediction horizon:", FDEs)
print('\n')

ADE for 2.0s prediction horizon: ['0.01823819', '0.12858750', '0.18331976', '0.11394396', '0.06975866', '0.05163631', '0.06204967', '0.01737141']
FDE for 2.0s prediction horizon: ['0.01774554', '0.25553013', '0.33623149', '0.19807822', '0.17270721', '0.07205833', '0.08351491', '0.01220816']




#### Constant Acceleration Model Result
ADE for 1.0s prediction horizon: ['0.01764429', '0.07241278', '0.10636568', '0.06684075', '0.03746524', '0.04254935', '0.05173001', '0.01834989']

FDE for 1.0s prediction horizon: ['0.00869598', '0.10830840', '0.17377816', '0.09861034', '0.02978359', '0.05538339', '0.06088321', '0.02041051']

ADE for 2.0s prediction horizon: ['0.01823819', '0.12858750', '0.18331976', '0.11394396', '0.06975866', '0.05163631', '0.06204967', '0.01737141']

FDE for 2.0s prediction horizon: ['0.01774554', '0.25553013', '0.33623149', '0.19807822', '0.17270721', '0.07205833', '0.08351491', '0.01220816']


ADE for 3.0s prediction horizon: ['0.01771143', '0.22354063', '0.27163772', '0.16077121', '0.13154264', '0.06007438', '0.06763599', '0.01850392']

FDE for 3.0s prediction horizon: ['0.01135669', '0.54843435', '0.52913740', '0.28482793', '0.32911254', '0.08568001', '0.07473506', '0.02742728']

Using Constant Turn Rate and Velocity(CTRV) model to predict the future (1s,2s,3s) trajectories of the target agent and calculate the ADE and FDE. 

In [15]:
# tranfer the xy acceleration to the linear and angular acceleration
def accelerationXY_to_nt(heading_angle_current,accelerationX,accelerationY):
    acceleration_n = accelerationX * math.sin(heading_angle_current) + accelerationY * math.cos(heading_angle_current)
    acceleration_t = accelerationX * math.cos(heading_angle_current) + accelerationY * math.sin(heading_angle_current)
    return acceleration_n, acceleration_t

tracks_gt_future = all_gt_future[tracks]
agents_to_predict = all_agent_trajs[tracks]

# acquire the initial heading angle
heading_angle_current = [0] * 8
heading_angle_current[:] = np.arctan(all_agent_current[tracks,7] / all_agent_current[tracks,8])

# acquire all the acceleration in the direction of X and Y
accelerationX = [[] for _ in range(8)]
accelerationY = [[] for _ in range(8)]
for num in range(8):
    accelerationX[num] = np.diff(agents_to_predict[num, 10 : 10 + predict_horizon + 1,7])
    accelerationY[num] = np.diff(agents_to_predict[num, 10 : 10 + predict_horizon + 1,8])

acceleration_n = []
acceleration_t = []
#initial acceleration n and t
for num in range(8):
    a_n = accelerationX[num][0] * np.sin(heading_angle_current[num]) + accelerationY[num][0] * np.cos(heading_angle_current[num])
    a_t = accelerationX[num][0] * np.cos(heading_angle_current[num]) + accelerationY[num][0] * np.sin(heading_angle_current[num])
    acceleration_n.append(a_n)
    acceleration_t.append(a_t)
#initial position and velocity
lastest_positionX = agents_to_predict[:,10,0]
lastest_positionY = agents_to_predict[:,10,1]
lastest_velocityX = agents_to_predict[:,10,7]
lastest_velocityY = agents_to_predict[:,10,8]

#initial Angular velocity
w = []
w = acceleration_n / np.sqrt(lastest_velocityX**2 + lastest_velocityY**2)

# initialization
timestep = 0.1
invalid = 0
predx = [[] for _ in range(8)]
predy = [[] for _ in range(8)]
newX = [0] * 8
newY = [0] * 8

# loop for each agent
for num in range(8):
    for t in range(predict_horizon):
        if agents_to_predict[num, 10 + t, -1] == 1:
            A = math.cos(heading_angle_current[num])
            B = math.sin(heading_angle_current[num])
            C = math.cos(heading_angle_current[num] + w[num] * timestep)
            D = math.sin(heading_angle_current[num] + w[num] * timestep)
            if invalid == 0:
                newX[num] = lastest_positionX[num] + timestep * lastest_velocityX[num] + (timestep * A / w[num] + B / w[num] ** 2 - D / w[num] ** 2) * acceleration_t[num] 
                + (A / w[num] ** 2 - timestep * B / w[num] - C / w[num] ** 2) * acceleration_n[num]

                lastest_velocityX[num] = lastest_velocityX[num] + (A / w[num] - C / w[num]) * acceleration_t[num] + (D / w[num] - B / w[num]) * acceleration_n[num]

                newY[num] = lastest_positionY[num] + timestep * lastest_velocityY[num] + (A / w[num]**2 - B * timestep / w[num] - C / w[num] ** 2) * acceleration_t[num]
                + (D / w[num] ** 2 - B / w[num] ** 2 - timestep * A / w[num]) * acceleration_n[num]

                lastest_velocityY[num] = lastest_velocityY[num] + (D / w[num] - B / w[num]) * acceleration_t[num] + (C / w[num] - A / w[num]) * acceleration_n[num]

                predx[num].append(newX[num])
                predy[num].append(newY[num])
                lastest_positionX[num], lastest_positionY[num] = newX[num], newY[num]

                heading_angle_current[num] = math.atan(lastest_velocityX[num] / lastest_velocityY[num])
                acceleration_t[num], acceleration_n[num] = accelerationXY_to_nt(heading_angle_current[num], accelerationX[num][t], accelerationY[num][t])


                w[num] = acceleration_n[num] / np.sqrt(lastest_velocityX[num] ** 2 + lastest_velocityY[num] ** 2)
                
            else:
                newX[num] = lastest_positionX[num] + timestep * (invalid + 1) * lastest_velocityX[num] + (timestep * (invalid + 1) * A / w[num] + B / w[num] ** 2 - D / w[num] ** 2) * acceleration_t[num] 
                + (A / w[num] ** 2 - timestep * (invalid + 1) * B / w[num] - C / w[num] ** 2) * acceleration_n[num]

                lastest_velocityX[num] = lastest_velocityX[num] + (A / w[num] - C / w[num]) * acceleration_t[num] + (D / w[num] - B / w[num]) * acceleration_n[num]

                newY[num] = lastest_positionY[num] + timestep * (invalid + 1) * lastest_velocityY[num] + (A / w[num]**2 - B * timestep * (invalid + 1) / w[num] - C / w[num] ** 2) * acceleration_t[num]
                + (D / w[num] ** 2 - B / w[num] ** 2 - timestep * (invalid + 1) * A / w[num]) * acceleration_n[num]

                lastest_velocityY[num] = lastest_velocityY[num] + (D / w[num] - B / w[num]) * acceleration_t[num] + (C / w[num] - A / w[num]) * acceleration_n[num]

                predx[num].append(newX[num])
                predy[num].append(newY[num])
                lastest_positionX[num], lastest_positionY[num] = newX[num], newY[num]

                heading_angle_current[num] = math.atan(lastest_velocityX[num] / lastest_velocityY[num])
                acceleration_t[num], acceleration_n[num] = accelerationXY_to_nt(heading_angle_current[num], accelerationX, accelerationY, num, t + invalid)

                if t <= predict_horizon - 1:
                    w[num] = acceleration_n[num] / np.sqrt(lastest_velocityX[num] ** 2 + lastest_velocityY[num] ** 2)
                
                invalid = 0
        else:
            invalid += 1

ADEs, FDEs = calculate_errors(predx, predy, tracks_gt_future, predict_horizon)

# Print results
print(f"ADE for {predict_horizon/10}s prediction horizon:", ADEs)
print(f"FDE for {predict_horizon/10}s prediction horizon:", FDEs)
print('\n')


ADE for 2.0s prediction horizon: ['0.14642424', '0.78044865', '1.45017797', '0.84587506', '0.37019182', '0.38004108', '0.52654446', '0.16298146']
FDE for 2.0s prediction horizon: ['0.29161816', '2.14419464', '3.85562840', '2.18635345', '0.94228398', '0.85300279', '1.14751881', '0.28472993']




#### CTRV Model Result
ADE for 1.0s prediction horizon: ['0.07161447', '0.28985375', '0.46309420', '0.27311693', '0.19619180', '0.17135041', '0.24604411', '0.08128108']

FDE for 1.0s prediction horizon: ['0.13643224', '0.52418152', '1.08428132', '0.64835616', '0.30339203', '0.34137912', '0.46491065', '0.16605214']

ADE for 2.0s prediction horizon: ['0.14642424', '0.78044865', '1.45017797', '0.84587506', '0.37019182', '0.38004108', '0.52654446', '0.16298146']

FDE for 2.0s prediction horizon: ['0.29161816', '2.14419464', '3.85562840', '2.18635345', '0.94228398', '0.85300279', '1.14751881', '0.28472993']


ADE for 3.0s prediction horizon: ['0.21776490', '1.90046228', '3.09152643', '1.71362117', '0.93137268', '0.66078994', '0.87272075', '0.21754978']

FDE for 3.0s prediction horizon: ['0.41285143', '6.18157398', '8.77249313', '4.60695817', '3.22242035', '1.55269063', '1.91117014', '0.35637455']

Briefly discuss the observations from your results. 

If you want to visualize the scenario here are some simple demo codes for you.

    'TYPE_FREEWAY': 1,
    'TYPE_SURFACE_STREET': 2,
    'TYPE_BIKE_LANE': 3,

    # for roadline
    'TYPE_UNKNOWN': -1,
    'TYPE_BROKEN_SINGLE_WHITE': 6,
    'TYPE_SOLID_SINGLE_WHITE': 7,
    'TYPE_SOLID_DOUBLE_WHITE': 8,
    'TYPE_BROKEN_SINGLE_YELLOW': 9,
    'TYPE_BROKEN_DOUBLE_YELLOW': 10,
    'TYPE_SOLID_SINGLE_YELLOW': 11,
    'TYPE_SOLID_DOUBLE_YELLOW': 12,
    'TYPE_PASSING_DOUBLE_YELLOW': 13,

    # for roadedge
    'TYPE_ROAD_EDGE_BOUNDARY': 15,
    'TYPE_ROAD_EDGE_MEDIAN': 16,

    # for stopsign
    'TYPE_STOP_SIGN': 17,

    # for crosswalk
    'TYPE_CROSSWALK': 18,

    # for speed bump
    'TYPE_SPEED_BUMP': 19,
    
    # for driveway
    'TYPE_DRIVEWAY': 20,

In [16]:
sdc_current_state = all_agent_trajs[tracks[0]][11]
ax = plt.gca()
fig = plt.gcf()
fig.set_facecolor('xkcd:grey') 
ax.set_facecolor('xkcd:grey')
for polyline in road_polylines:
    map_type = polyline[0,6]
    if map_type == 6:
        plt.plot(polyline[:, 0], polyline[:, 1], 'w', linestyle='dashed', linewidth=1)
    elif map_type == 7:
        plt.plot(polyline[:, 0], polyline[:, 1], 'w', linestyle='solid', linewidth=1)
    elif map_type == 8:
        plt.plot(polyline[:, 0], polyline[:, 1], 'w', linestyle='solid', linewidth=1)
    elif map_type == 9:
        plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='dashed', linewidth=1)
    elif map_type == 10:
        plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='dashed', linewidth=1)
    elif map_type == 11:
        plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='solid', linewidth=1)
    elif map_type == 12:
        plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='solid', linewidth=1)
    elif map_type == 13:
        plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='dotted', linewidth=1)
    elif map_type == 15:
        plt.plot(polyline[:, 0], polyline[:, 1], 'k', linewidth=1)
    elif map_type == 16:
        plt.plot(polyline[:, 0], polyline[:, 1], 'k', linewidth=1)
ax.axis([-70+ sdc_current_state[0], 70+ sdc_current_state[0], -70+ sdc_current_state[1], 70 + sdc_current_state[1]])

filename = './viz.png' 
plt.savefig(filename)       
plt.close()